In [2]:
!pip install pyaudio

In [3]:
import pyaudio

p = pyaudio.PyAudio()

print("=== Danh sách thiết bị âm thanh ===\n")
for i in range(p.get_device_count()):
    info = p.get_device_info_by_index(i)
    # Chỉ hiển thị thiết bị có input channels (microphone)
    if info['maxInputChannels'] > 0:
        print(f"Device {i}: {info['name']}")
        print(f"  - Input Channels: {info['maxInputChannels']}")
        print(f"  - Sample Rate: {int(info['defaultSampleRate'])} Hz")
        print(f"  - Host API: {p.get_host_api_info_by_index(info['hostApi'])['name']}")
        print()

p.terminate()

=== Danh sách thiết bị âm thanh ===

Device 0: Microsoft Sound Mapper - Input
  - Input Channels: 2
  - Sample Rate: 44100 Hz
  - Host API: MME

Device 1: Microphone (MAONO AU-BM10)
  - Input Channels: 1
  - Sample Rate: 44100 Hz
  - Host API: MME

Device 2: Microphone Array (Intel® Smart 
  - Input Channels: 4
  - Sample Rate: 44100 Hz
  - Host API: MME

Device 6: Primary Sound Capture Driver
  - Input Channels: 2
  - Sample Rate: 44100 Hz
  - Host API: Windows DirectSound

Device 7: Microphone (MAONO AU-BM10)
  - Input Channels: 1
  - Sample Rate: 44100 Hz
  - Host API: Windows DirectSound

Device 8: Microphone Array (Intel® Smart Sound Technology for Digital Microphones)
  - Input Channels: 4
  - Sample Rate: 44100 Hz
  - Host API: Windows DirectSound

Device 14: Microphone Array (Intel® Smart Sound Technology for Digital Microphones)
  - Input Channels: 2
  - Sample Rate: 48000 Hz
  - Host API: Windows WASAPI

Device 15: Microphone (MAONO AU-BM10)
  - Input Channels: 1
  - Sample R

In [6]:
#Test ghi âm 5 với Mic MAONO xem mic hoạt động không 
"""
Script để ghi âm từ microphone MAONO AU-BM10 (Device 0)
Ghi âm 5 giây và lưu vào thư mục data/audios
Sử dụng sounddevice để tương thích tốt hơn với ALSA trên Linux
"""

import sounddevice as sd
import soundfile as sf
import numpy as np
import os
from datetime import datetime

# Cấu hình
DEVICE_INDEX = 0  # MAONO AU-BM10: USB Audio (hw:0,0)
CHANNELS = 1  # Mono
SAMPLE_RATE = 44100  # Hz
RECORD_SECONDS = 5  # Thời gian ghi âm
OUTPUT_DIR = "data/audios"

def main():
    # Tạo tên file với timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = os.path.join(OUTPUT_DIR, f"recording_{timestamp}.wav")
    
    # Tạo thư mục nếu chưa tồn tại
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print(f"Bắt đầu ghi âm từ Device {DEVICE_INDEX}: MAONO AU-BM10")
    print(f"Thời gian ghi: {RECORD_SECONDS} giây")
    print(f"Đang ghi âm...")
    
    try:
        # Ghi âm với sounddevice
        recording = sd.rec(
            int(RECORD_SECONDS * SAMPLE_RATE),
            samplerate=SAMPLE_RATE,
            channels=CHANNELS,
            device=DEVICE_INDEX,
            dtype='int16'
        )
        
        # Chờ ghi âm hoàn thành
        sd.wait()
        
        print(f"✓ Hoàn thành ghi âm!")
        
        # Lưu file WAV
        sf.write(output_file, recording, SAMPLE_RATE, subtype='PCM_16')
        
        print(f"✓ File đã được lưu tại: {output_file}")
        print(f"  - Kích thước: {os.path.getsize(output_file) / 1024:.2f} KB")
        print(f"  - Định dạng: WAV, {SAMPLE_RATE} Hz, {CHANNELS} channel(s), 16-bit PCM")
        
    except Exception as e:
        print(f"✗ Lỗi: {e}")
        print("\nThử kiểm tra lại device index bằng lệnh:")
        print("python -c \"import sounddevice as sd; print(sd.query_devices())\"")

if __name__ == "__main__":
    main()


Bắt đầu ghi âm từ Device 0: MAONO AU-BM10
Thời gian ghi: 5 giây
Đang ghi âm...
✓ Hoàn thành ghi âm!
✓ File đã được lưu tại: data/audios\recording_20260128_143324.wav
  - Kích thước: 430.71 KB
  - Định dạng: WAV, 44100 Hz, 1 channel(s), 16-bit PCM


In [4]:
!pip install soundfile

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 1.3 MB/s eta 0:00:01
   ------------------------------ --------- 0.8/1.0 MB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 1.3 MB/s eta 0:00:00


In [ ]:
#Test Gửi WAV lên Server nhận WAV về
import requests
import subprocess
import os
import json

# ==============================
# Cấu hình (Python tự set eth0 & gain)
# ==============================
speaker_bin = "/home/unitree/UnitreeG1Meiko-AI_Conversation-LLM-RAG/unitree_sdk2/build/bin/speaker_test"
network_interface = "eth0"
gain = "3.0"  # Python tự set, không nhập từ terminal

url = "https://willow-pseudomodern-griffin.ngrok-free.dev/process_audio"
input_file_path = "/home/unitree/UnitreeG1Meiko-AI_Conversation-LLM-RAG/data/audios/7.wav"
output_file_name = "output.wav"

# ==============================
# MAIN
# ==============================
try:
    # 1. Gửi request tới server
    with open(input_file_path, "rb") as f:
        files = {"audio": ("input.wav", f, "audio/wav")}
        print(f"Đang gửi request tới {url}...")
        resp = requests.post(url, files=files)

    print("Status:", resp.status_code)

    if resp.status_code == 200:
        # 2. Lưu file trả về
        abs_output_path = os.path.abspath(output_file_name)
        with open(abs_output_path, "wb") as out:
            out.write(resp.content)

        print(f"Đã lưu file tại: {abs_output_path}")

        # 3. Chạy speaker_test với đúng format mới
        # FORMAT: ./speaker_test eth0 audio.wav gain
        cmd = [speaker_bin, network_interface, abs_output_path, gain]

        print(f"Đang chạy lệnh: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode == 0:
            print("Phát âm thanh thành công!")
            print(result.stdout)
        else:
            print("Lỗi khi phát âm thanh:")
            print(result.stderr)

    else:
        print("Lỗi request:", resp.text)

except Exception as e:
    print(f"Có lỗi xảy ra: {e}")


Đang gửi request tới https://willow-pseudomodern-griffin.ngrok-free.dev/process_audio...
Status: 404
Lỗi request: <!DOCTYPE html>
<html class="h-full" lang="en-US" dir="ltr">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="preload" href="https://assets.ngrok.com/fonts/euclid-square/EuclidSquare-Regular-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://assets.ngrok.com/fonts/euclid-square/EuclidSquare-RegularItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://assets.ngrok.com/fonts/euclid-square/EuclidSquare-Medium-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://assets.ngrok.com/fonts/euclid-square/EuclidSquare-MediumItalic-WebS.woff" as="font" type="font/woff" crossorigin="anonymous" />
    <link rel="preload" href="https://assets.ngrok.com/fonts/i

In [ ]:
#Test nhận diện keyword Xin chào qua mic
import os
import sys

# =====================
# TẮT LOG VOSK + KALDI
# =====================
os.environ["VOSK_LOG_LEVEL"] = "-1"       # -1 = disable all logs
os.environ["KALDI_DISABLE_LOGGING"] = "1" # disable kaldi logging
sys.stderr = open(os.devnull, 'w')        # redirect stderr để không in ra console

import time
import json
import wave
import queue
import sounddevice as sd
import numpy as np
from vosk import Model, KaldiRecognizer

# =====================
# CẤU HÌNH
# =====================
DEVICE_INDEX = 0
MIC_RATE = 44100
VOSK_RATE = 16000
CHANNELS = 1
TIMEOUT = 10
WAKEWORD = ["xin chao", "xin chào"]
DEBUG_WAV = "capture_debug.wav"
MODEL_PATH = "vosk-model-vn-0.4"

# =====================
# KHỞI TẠO MODEL
# =====================
print("📦 Đang load mô hình Vosk (VN)...")
model = Model(MODEL_PATH)
recognizer = KaldiRecognizer(model, VOSK_RATE)

# =====================
# QUEUE AUDIO
# =====================
audio_q = queue.Queue()
frames = []
detected = False

# =====================
# CALLBACK AUDIO
# =====================
def audio_callback(indata, frames_count, time_info, status):
    if status:
        print("⚠️", status)
    audio_q.put(indata.copy())

# =====================
# MỞ STREAM
# =====================
print("🎤 Đang mở micro device 0...")
stream = sd.InputStream(
    samplerate=MIC_RATE,
    channels=CHANNELS,
    dtype='int16',
    device=DEVICE_INDEX,
    blocksize=1024,
    callback=audio_callback
)

stream.start()
print(f"🟢 Nói 'Xin chào' trong vòng {TIMEOUT} giây...")

start = time.time()

# =====================
# VÒNG XỬ LÝ CHÍNH
# =====================
while True:
    if time.time() - start > TIMEOUT:
        print("⏹ Timeout — không phát hiện.")
        break

    try:
        data = audio_q.get_nowait()
    except queue.Empty:
        time.sleep(0.005)  # tránh 100% CPU
        continue

    frames.append(data)

    # Resample 44100 -> 16000
    pcm = data[:, 0].astype(np.int16)
    factor = MIC_RATE // VOSK_RATE
    pcm16 = pcm[::factor].tobytes()

    if recognizer.AcceptWaveform(pcm16):
        text = json.loads(recognizer.Result()).get("text", "").lower()
        if text:
            print("🗣", text)
        if any(w in text for w in WAKEWORD):
            print("🎯 PHÁT HIỆN WAKEWORD!")
            detected = True
            break

stream.stop()
stream.close()

# =====================
# LƯU WAV DEBUG
# =====================
print("💾 Đang lưu debug WAV...")

with wave.open(DEBUG_WAV, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(2)
    wf.setframerate(MIC_RATE)
    wf.writeframes(b''.join([f.tobytes() for f in frames]))

print(f"📁 Đã lưu file: {DEBUG_WAV}")

# =====================
# KẾT QUẢ
# =====================
if detected:
    print("🟢 KẾT QUẢ: Đã tìm thấy 'Xin chào'")
else:
    print("🔴 KẾT QUẢ: Không tìm thấy wakeword.")

print("🛑 Dừng.")


📦 Đang load mô hình Vosk (VN)...
🎤 Đang mở micro device 0...
🟢 Nói 'Xin chào' trong vòng 10 giây...
🗣 xin chào
🎯 PHÁT HIỆN WAKEWORD!
💾 Đang lưu debug WAV...
📁 Đã lưu file: capture_debug.wav
🟢 KẾT QUẢ: Đã tìm thấy 'Xin chào'
🛑 Dừng.


In [ ]:
import sounddevice as sd
import soundfile as sf
import numpy as np
import os
import requests
from datetime import datetime
from IPython.display import Audio, display

# ==============================
# CONFIG GHI ÂM
# ==============================
def find_maono_device():
    devices = sd.query_devices()
    for i, dev in enumerate(devices):
        if "MAONO" in dev['name'] and dev['max_input_channels'] > 0:
            return i
    return 0  # fallback

DEVICE_INDEX = find_maono_device()
CHANNELS = 1
SAMPLE_RATE = 44100
RECORD_SECONDS = 5

ROOT_DIR = os.path.dirname(os.getcwd())
OUTPUT_DIR = os.path.join(ROOT_DIR, "data", "audios")

# ==============================
# CONFIG SERVER
# ==============================
url = "https://semiphenomenal-camdyn-squirrely.ngrok-free.dev/process_audio"

# Thư mục mới để lưu file trả về
RETURN_DIR = os.path.join(ROOT_DIR, "data", "audiocases_rep")

# ==============================
# MAIN FLOW
# ==============================
def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(RETURN_DIR, exist_ok=True)

    # ---- 1. RECORD ----
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    local_file = os.path.join(OUTPUT_DIR, f"recording_{timestamp}.wav")

    print(f"[GHI ÂM] Device: {sd.query_devices(DEVICE_INDEX)['name']}")
    print(f"[GHI ÂM] {RECORD_SECONDS}s... hãy nói vào mic")
    
    try:
        recording = sd.rec(
            int(RECORD_SECONDS * SAMPLE_RATE),
            samplerate=SAMPLE_RATE,
            channels=CHANNELS,
            device=DEVICE_INDEX,
            dtype='int16'
        )
        sd.wait()
        sf.write(local_file, recording, SAMPLE_RATE, subtype='PCM_16')

        print(f"[OK] Lưu xong: {local_file}")
        display(Audio(local_file))

    except Exception as e:
        print(f"[ERR] Lỗi ghi âm: {e}")
        print(sd.query_devices())
        return
    
    # ---- 2. SEND TO SERVER ----
    print(f"[REQ] Đang gửi WAV tới server: {url}")
    try:
        with open(local_file, "rb") as f:
            files = {"audio": ("input.wav", f, "audio/wav")}
            resp = requests.post(url, files=files)

        print(f"[RESP] Status: {resp.status_code}")

        if resp.status_code != 200:
            print("[ERR] Server trả lỗi:", resp.text)
            return

    except Exception as e:
        print(f"[ERR] Lỗi gửi request: {e}")
        return

    # ---- 3. SAVE RETURNED WAV ----
    try:
        # tạo tên file trả về có timestamp
        return_file = os.path.join(RETURN_DIR, f"server_output_{timestamp}.wav")

        with open(return_file, "wb") as out:
            out.write(resp.content)

        print(f"[OK] Đã lưu file server trả về: {return_file}")
        display(Audio(return_file))

    except Exception as e:
        print(f"[ERR] Không lưu được file server trả về: {e}")

if __name__ == "__main__":
    main()


In [ ]:
import sounddevice as sd
import numpy as np
import time

# ======================
# CONFIG
# ======================
DEVICE_INDEX = 1
SAMPLE_RATE = 16000
CHANNELS = 1
FRAME_DURATION = 0.2              # mỗi buffer dài 0.2s
FRAME_SIZE = int(SAMPLE_RATE * FRAME_DURATION)

# NGƯỠNG PHÁT HIỆN
THRESHOLD = 300                   # biên độ (có thể chỉnh)
SILENCE_DURATION = 1.0            # giây im lặng để coi là kết thúc câu

# ======================
# STATE
# ======================
last_voice_time = time.time()
is_speaking = False

print("🎤 Bắt đầu lắng nghe micro ...")
print(f"🔧 DEVICE={DEVICE_INDEX}, THRESHOLD={THRESHOLD}, SILENCE_DURATION={SILENCE_DURATION}s")

try:
    with sd.InputStream(samplerate=SAMPLE_RATE, channels=CHANNELS, dtype='int16', device=DEVICE_INDEX) as stream:
        while True:
            audio = stream.read(FRAME_SIZE)[0]
            amp = np.abs(audio).mean()     # giá trị biên độ trung bình

            now = time.time()

            # ======================
            # DEBUG LOG
            # ======================
            print(f"[DEBUG] AMP={amp:.1f}, speaking={is_speaking}, silence={now - last_voice_time:.2f}s")

            # ======================
            # PHÁT HIỆN NÓI
            # ======================
            if amp > THRESHOLD:
                if not is_speaking:
                    print("🟢 BẮT ĐẦU NÓI")
                is_speaking = True
                last_voice_time = now
            else:
                # Nếu đang nói mà im lặng lâu => kết thúc câu
                if is_speaking and (now - last_voice_time > SILENCE_DURATION):
                    print("🔴 KẾT THÚC CÂU!")
                    is_speaking = False

            time.sleep(0.01)

except KeyboardInterrupt:
    print("🛑 Dừng lắng nghe.")
